In [4]:
#Importaciones necesarias
import re  
import json
import nltk
import time
import pickle
import numpy as np

from datetime import datetime
from nltk.stem import SnowballStemmer
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from selenium.webdriver.common.by import By
from tensorflow.keras.models import load_model
from selenium.webdriver.common.keys import Keys   
from tensorflow.keras.layers import Dense, Dropout

In [5]:
ignorar = ['!','¡','¿','?','.']
stemmer = SnowballStemmer('spanish')
model = Sequential()

In [6]:
class Func():
    
    def __init__(self)-> None:
        pass
    
    def Json(self, data, filename):
        self.data = data
        archivo = open(filename,'w')
        json.dump(data, archivo, indent=4)
        
    def Modelo(self, mod):        
        self.model = mod
        
    def Final(self, modelo, tags, palabras):
        self.modelo = modelo
        self.tags = tags
        self.palabras = palabras
        
    def cleanEntrada(self, ent_usuario):
        ent_usuario = nltk.word_tokenize(ent_usuario)
        ent_usuario = [stemmer.stem(w.lower()) for w in ent_usuario if w not in ignorar]
        return ent_usuario

    def convVector(self, ent_usuario):    
        ent_usuario = self.cleanEntrada(ent_usuario)    
        vect_entrada = [0]*len(self.palabras) 
        for palabra in ent_usuario:              
            if palabra in self.palabras:          
                indice = self.palabras.index(palabra)
                vect_entrada[indice] = 1                 
        vect_entrada = np.array(vect_entrada)           
        return vect_entrada

    def getTags(self, vect_entrada, LIMITE = 0):
        vect_salida = self.model.predict(np.array([vect_entrada]))[0]
        vect_salida = [[i,r] for i,r in enumerate(vect_salida) if r > LIMITE]
        vect_salida.sort(key=lambda x: x[1], reverse=True)
        print(vect_salida)    
        list_etiquetas = []    
        for r in vect_salida:   
            list_etiquetas.append({"intent": self.tags[r[0]], "probability": str(r[1])})   
        return list_etiquetas

    def getResponse(self, list_etiquetas):
        etiqueta = list_etiquetas[0]['intent']
        list_data = self.data['intents']
        for dat in list_data:
            if etiqueta == dat['tag']:
                list_respuestas = dat['resp']            
                break
        return list_respuestas

    def chatbotRespuesta(self, ent_usuario):
        vect_entrada = self.convVector(ent_usuario)
        list_etiquetas = self.getTags(vect_entrada, LIMITE = 0)
        respuesta = self.getResponse(list_etiquetas)
        return respuesta


In [14]:
obj = Func()
model = Sequential()

class Chat():
    #Creacion del banco de palabras
    dias_pres = ['28/07/2023','30/07/2023']
    dias_virt = ['26/07/2023','27/07/2023','31/07/2023']
    data = { 'intents':[{'tag':'Saludo',
                         'preg':['hola', 'holaa', 'holaaa', 'buenos dias', 'buenas tardes', 'buenas noches', 'buenas'],
                         'resp':['Hola!, soy NabiBot y estoy aqui para ayudarte',
                                 'Selecciona una opcion escribiendo una de las palabras que aparece a continuacion',
                                 '',
                                 '*Menu*',
                                 '*1.* Agendar cita',
                                 '*2.* Servicio de envio',
                                 '*3.* Cotizacion de presupuesto']},
                        
                        {'tag':'Agendar_cita',
                         'preg':['agendar cita', 'programar cita', 'programar una cita', 'reservar una reunion', 'acordar una reunion'],
                         'resp':['*Agendar Cita*',
                                 'Le gustaria que si cita fuese',
                                 '*1.* Virtual',
                                 '*2.* Presencial']},                         
                        
                        {'tag':'Cita_virtual',
                         'preg':['cita virtual','virtual','reservar cita virtual'],
                         'resp':['*Cita virtual*',
                                 'Para este mes tenemos disponibilidad los días',
                                 f'{", ".join(dias_virt)}',
                                 '',
                                 '_Todas nuestras citas son a las 3 p.m. por zoom_.',
                                 'Se le hará llegar el link 10 minutos antes el día de la reunión',
                                 '',
                                 '*Porfavor seleccione una fecha en formato dd/mm/yyyy*']}, 
                        
                        {'tag':'Cita_presencial',
                         'preg':['cita prescencial','cita presencial','presencial','prescencial','reservar cita presencial'],
                         'resp':['*Cita Presencial*',
                                 'Para este mes tenemos disponibilidad los siguientes días',
                                 f'{", ".join(dias_pres)}',
                                 '',
                                 'Porfavor seleccione una fecha en formato dd/mm/yyyy',
                                 '',
                                 'Todas nuestras reuniones presenciales son a las 3 p.m. en nuestras oficinas principales',
                                 'Google Maps: https://goo.gl/maps/rX1cvabSMZhjmPvM9']}, 
                        
                        {'tag':'Servicio_envio',
                         'preg':['servicio de envio','como puedo realizar un envio', 'solicitud de envios','enviar paquetes','como envio un paquete', 'envio'],
                         'resp':['*Servicio de Envio*',
                                 '*1.* Nacional (Panama)',
                                 '*2.* Internacional (America)',
                                 '*3.* Internacional (Europa, Asia, Oceania)',
                                 '',
                                 'Seleccione el destino del paquete']}, 
                        
                        {'tag':'Nacional_panama',
                         'preg':['nacional'],
                         'resp':['*Nacional (Panama)*',
                                 '*1.* Bocas del Toro',
                                 '*2.* Coclé',
                                 '*3.* Colón',
                                 '*4.* Chiriquí',
                                 '*5.* Darién',
                                 '*6.* Herrera',
                                 '*7.* Los Santos',
                                 '*8.* Panamá (Capital)',
                                 '*9.* Veraguas',
                                 '*10.* Panamá Oeste',
                                 '*11.* Comarcas',
                                 '',
                                 'Seleccione el destino del paquete.']},
                        
                        {'tag':'Bocas_provincia',
                         'preg':['bocas del toro'],
                         'resp':['*Bocas del Toro*',
                                 'Brinde la siguiente informacion del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Cocle_provincia',
                         'preg':['Cocle', 'Coclé'],
                         'resp':['*Coclé*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},
                        
                        {'tag':'Colon_provincia',
                         'preg':['Colon', 'Colón'],
                         'resp':['*Colón*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Chiriqui_provincia',
                         'preg':['Chiriqui', 'Chiriquí'],
                         'resp':['*Chiriquí*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Darien_provincia',
                         'preg':['Darien', 'Darién'],
                         'resp':['*Darién*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Herrera_provincia',
                         'preg':['Herrera'],
                         'resp':['*Herrera*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Los_Santos_provincia',
                         'preg':['Los Santos'],
                         'resp':['*Los Santos*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Panama_provincia',
                         'preg':['Panama', 'Panamá'],
                         'resp':['*Panamá (capital)*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Veraguas_provincia',
                         'preg':['Veraguas'],
                         'resp':['*Veraguas*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Panama_Oeste_provincia',
                         'preg':['Panamá Oeste', 'Panama Oeste'],
                         'resp':['*Panamá Oeste*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 20kg*',
                                 '*2. 10kg*']},

                        {'tag':'Comarcas_provincia',
                         'preg':['Comarca', 'Comarcas'],
                         'resp':['*Comarcas*',
                                 'Brinde la siguiente información del envio en el orden solicitado',
                                 'Nombre del destinatario',
                                 'Cédula del destinatario',
                                 'Cantidad de paquetes con su peso',
                                 '*EJEMPLO*',
                                 '*1. 5 paquete, 20kg*',
                                 '*2. 1 paquete, 10kg*']},


                        {'tag':'Infor_recibida',
                         'preg':['Juan Vela','1-234-5678','1 paquete, 5kg'],
                         'resp':['*Información recibida*',
                                 'Tus datos son:',
                                 'Nombre del destinatario: Juan Vela',
                                 'Cédula del destinatario: 1-234-5678',
                                 'Cantidad y peso del paquete: 1 paquete, 5kg',
                                 'Gracias por elegir a SIC-Panamá']},
                        
                        {'tag':'Cotizacion_presupuesto',
                         'preg':['cotizacion ', 'cotizacion del presupuesto', 'cotizar'],
                         'resp':['*Cotización de presupuesto*',
                                 'Puede proporcionarme los detalles del envío en el orden solicitado:',
                                 'Origen del envío',
                                 'Destino del envío',
                                 'Peso del paquete',
                                 'Dimensiones en mts (largo, alto, ancho)']},
                        
                         {'tag':'Cotizacion_recibida',
                         'preg':['Capital', 'Bocas', '10kg', '0.5, 0.5, 0.2'],
                         'resp':['*Cotizacion recibida*',
                                 'Costo neto de envío de Panamá(Capital) a Bocas del Toro: $7',
                                 '+ peso de 10kg: $17',
                                 '+ costos adicionales: $3',
                                 '*TOTAL:* $20.00']},

                        {'tag':'contacto',
                         'preg':['Contacto','Ubicación', 'Contactos'],
                         'resp':['*Contactos*',
                                 '',
                                 'Ubicación: Parque Natural Metropolitano, STEAM House, Rep. de Panamá, Av. Juan Pablo II 7, Panama City',
                                 'Google maps: https://goo.gl/maps/rX1cvabSMZhjmPvM9',
                                 'Horario: Lunes a Sábado 8:00 a.m - 5:00 p.m',
                                 'Teléfono: +507 300-0000']},
                
                        {'tag':'Despedida',
                         'preg':['chao', 'adios', 'hasta luego','gracias','muchas gracias'],
                         'resp':['¡Hasta luego!, espero poder estar en contacto denuevo contigo.']},
                        
                        {'tag':'Salida',
                         'preg':['exit'],
                         'resp':['sali con exit-o']},
                        
                        {'tag':'No_resp',
                         'preg':[''],
                         'resp':['No he logrado entender lo que me escribiste',
                                 '',
                                 'Porfavor revisa lo que has escrito e intenta hacerme la consulta nuevamente']}
    ]}
    obj.Json(data,'intenciones.json')
    obj.Modelo(model)
    
    palabras = []
    tags = []
    documents = []
        
    for intents in data['intents']:
        tags.append(intents['tag'])
        for preguntas in intents['preg']:
            preguntas_res = nltk.word_tokenize(preguntas)
            palabras.extend(preguntas_res) 
            documents.append((preguntas_res, intents['tag']))
    
    palabras = [stemmer.stem(w.lower()) for w in palabras if w not in ignorar]
    
    pickle.dump(palabras, open('palabras.pkl', 'wb'))
    pickle.dump(tags, open('tags.pkl', 'wb'))
    
    entrenamiento = []
    
    for doc in documents:
        inter = doc[0]
        inter = [stemmer.stem(w.lower()) for w in inter if w not in ignorar]
        ent_cod = []
        for pal in palabras:
            if pal in inter:
                ent_cod.append(1)
            else:
                ent_cod.append(0)
        sal_cod = [0]*len(tags)
        indice = tags.index(doc[1])
        sal_cod[indice] = 1
        entrenamiento.append([ent_cod, sal_cod])
    
    entrenamiento = np.array(entrenamiento, dtype = list)
    x_train = list(entrenamiento[:,0])
    y_train = list(entrenamiento[:,1])
    
    model.add(Dense(128, input_shape=(len(x_train[0]),), activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64,activation = 'relu')) 
    model.add(Dropout(0.5))
    model.add(Dense(len(y_train[0]),activation = 'softmax'))

    sgd = SGD(learning_rate=0.01,momentum=0.9,nesterov=True) 
    model.compile(loss = "categorical_crossentropy",optimizer = sgd, metrics = ["accuracy"])

    hist = model.fit(np.array(x_train),np.array(y_train),epochs = 300,batch_size = 10,verbose = True)
    model.save("modelo_chatbot.h5", hist)
    
    print("##### MODELO CREADO #####")
    
    modelo = load_model('modelo_chatbot.h5')
    data = json.loads(open('intenciones.json').read())
    palabras = pickle.load(open("palabras.pkl","rb"))
    tags = pickle.load(open("tags.pkl","rb"))
    
    obj.Final(modelo, tags, palabras)

Epoch 1/300
7/7 [==============================] - 1s 5ms/step - loss: 3.2117 - accuracy: 0.0462
Epoch 2/300
7/7 [==============================] - 0s 9ms/step - loss: 3.1958 - accuracy: 0.0769
Epoch 3/300
7/7 [==============================] - 0s 12ms/step - loss: 3.0778 - accuracy: 0.1538
Epoch 4/300
7/7 [==============================] - 0s 4ms/step - loss: 3.0952 - accuracy: 0.1231
Epoch 5/300
7/7 [==============================] - 0s 5ms/step - loss: 2.9970 - accuracy: 0.2000
Epoch 6/300
7/7 [==============================] - 0s 3ms/step - loss: 2.9214 - accuracy: 0.2308
Epoch 7/300
7/7 [==============================] - 0s 3ms/step - loss: 2.7999 - accuracy: 0.2154
Epoch 8/300
7/7 [==============================] - 0s 11ms/step - loss: 2.7823 - accuracy: 0.2000
Epoch 9/300
7/7 [==============================] - 0s 3ms/step - loss: 2.6675 - accuracy: 0.1846
Epoch 10/300
7/7 [==============================] - 0s 3ms/step - loss: 2.5303 - accuracy: 0.3231
Epoch 11/300
7/7 [=========

7/7 [==============================] - 0s 3ms/step - loss: 0.4687 - accuracy: 0.9077
Epoch 85/300
7/7 [==============================] - 0s 3ms/step - loss: 0.4360 - accuracy: 0.8923
Epoch 86/300
7/7 [==============================] - 0s 9ms/step - loss: 0.4436 - accuracy: 0.8615
Epoch 87/300
7/7 [==============================] - 0s 5ms/step - loss: 0.4448 - accuracy: 0.9385
Epoch 88/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4041 - accuracy: 0.9231
Epoch 89/300
7/7 [==============================] - 0s 3ms/step - loss: 0.4095 - accuracy: 0.9077
Epoch 90/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4896 - accuracy: 0.9077
Epoch 91/300
7/7 [==============================] - 0s 4ms/step - loss: 0.3399 - accuracy: 0.9231
Epoch 92/300
7/7 [==============================] - 0s 5ms/step - loss: 0.3451 - accuracy: 0.9385
Epoch 93/300
7/7 [==============================] - 0s 4ms/step - loss: 0.6189 - accuracy: 0.8308
Epoch 94/300
7/7 [===============

7/7 [==============================] - 0s 7ms/step - loss: 0.1428 - accuracy: 0.9846
Epoch 168/300
7/7 [==============================] - 0s 11ms/step - loss: 0.2146 - accuracy: 0.9538
Epoch 169/300
7/7 [==============================] - 0s 3ms/step - loss: 0.2804 - accuracy: 0.8923
Epoch 170/300
7/7 [==============================] - 0s 3ms/step - loss: 0.2081 - accuracy: 0.9538
Epoch 171/300
7/7 [==============================] - 0s 10ms/step - loss: 0.1842 - accuracy: 0.9692
Epoch 172/300
7/7 [==============================] - 0s 3ms/step - loss: 0.2096 - accuracy: 0.9538
Epoch 173/300
7/7 [==============================] - 0s 4ms/step - loss: 0.3154 - accuracy: 0.8462
Epoch 174/300
7/7 [==============================] - 0s 8ms/step - loss: 0.2014 - accuracy: 0.9692
Epoch 175/300
7/7 [==============================] - 0s 3ms/step - loss: 0.2279 - accuracy: 0.9077
Epoch 176/300
7/7 [==============================] - 0s 4ms/step - loss: 0.2371 - accuracy: 0.9385
Epoch 177/300
7/7 [===

7/7 [==============================] - 0s 11ms/step - loss: 0.1005 - accuracy: 0.9846
Epoch 250/300
7/7 [==============================] - 0s 4ms/step - loss: 0.1801 - accuracy: 0.9538
Epoch 251/300
7/7 [==============================] - 0s 4ms/step - loss: 0.1054 - accuracy: 1.0000
Epoch 252/300
7/7 [==============================] - 0s 4ms/step - loss: 0.1080 - accuracy: 0.9692
Epoch 253/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0376 - accuracy: 1.0000
Epoch 254/300
7/7 [==============================] - 0s 9ms/step - loss: 0.1524 - accuracy: 0.9538
Epoch 255/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0919 - accuracy: 0.9846
Epoch 256/300
7/7 [==============================] - 0s 3ms/step - loss: 0.1363 - accuracy: 0.9846
Epoch 257/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0763 - accuracy: 1.0000
Epoch 258/300
7/7 [==============================] - 0s 5ms/step - loss: 0.1193 - accuracy: 0.9846
Epoch 259/300
7/7 [===

In [10]:
#Importacion de selenium para abrir el software automatizado en Edge
from selenium import webdriver
driver = webdriver.Edge()
driver.get("https://web.whatsapp.com/")

In [ ]:
def Verificar(usuario):
    try:
        num_mens = usuario.find_element(By.CLASS_NAME,"_1pJ9J").text
        mens_leer = re.findall('\d+' ,num_mens)
        if len(mens_leer) != 0:
            pendiente = True
        else:
            pendiente = False
    except:
        pendiente = False
    return pendiente

def obtenerMensaje(chat):
    mensaje = chat.find_element(By.CLASS_NAME,"_21Ahp").text
    html = chat.get_attribute('outerHTML')
    persona = re.findall(r'data-pre-plain-text=.{1}\[.+].(.+?):', html)
    return persona, mensaje

usuario_men = ''

while usuario_men != 'salir':
    print('#', end = " ")
    time.sleep(1)
    conversaciones = driver.find_elements(By.CLASS_NAME, "_1Oe6M") 
    
    for usuario in conversaciones:
        nombres = usuario.find_element(By.CLASS_NAME,"_21S-L")
        no_resp = Verificar(usuario)
        
        if no_resp:
            conversacion = usuario.find_element(By.CLASS_NAME,"vQ0w7")
            conversacion.click()  
            time.sleep(1)        
            chats = driver.find_elements(By.CLASS_NAME,"_27K43")     
  
            for chat in chats[-4:]:
                persona, mensaje = obtenerMensaje(chat)
                usuario_men = mensaje.lower()                     
                
            print("Usuario:",persona, "Mensaje:", usuario_men, end = " ")
            respuesta = driver.find_element(By.CLASS_NAME,"_3Uu1_")              
            bot_resp = obj.chatbotRespuesta(usuario_men)        
            
            for res in bot_resp:
                respuesta.send_keys(res, Keys.SHIFT + Keys.ENTER)          
            respuesta.send_keys(Keys.ENTER)                 
            time.sleep(1)

            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()    

# # # # # # # # # # # # # Usuario: ['NabiBot'] Mensaje: hola [[0, 0.9987287], [23, 0.001134546], [22, 5.6677876e-05], [6, 2.4407465e-05], [18, 1.277076e-05], [11, 9.341212e-06], [20, 8.32464e-06], [4, 4.9668984e-06], [8, 3.8799176e-06], [10, 3.4951725e-06], [9, 2.0853656e-06], [19, 1.9027694e-06], [5, 1.631648e-06], [3, 1.342837e-06], [7, 1.088389e-06], [14, 1.0446113e-06], [12, 9.004416e-07], [2, 8.560413e-07], [16, 7.4356075e-07], [17, 5.497412e-07], [1, 5.2438463e-07], [21, 4.3183e-08], [13, 3.4956457e-08], [15, 2.4622725e-08]]
# # # # # Usuario: ['NabiBot'] Mensaje: servicio de envio [[4, 0.96574], [23, 0.014488988], [17, 0.008702961], [0, 0.004555763], [20, 0.0013680395], [7, 0.0006282937], [11, 0.00059280277], [10, 0.0005754182], [12, 0.0005716475], [22, 0.00052177516], [15, 0.00043608213], [8, 0.00039569684], [21, 0.00027298316], [5, 0.00026877632], [9, 0.00025969063], [13, 0.00018491287], [19, 0.00014453009], [6, 7.889367e-05], [2, 6.8955094e-05], [1, 6.318214e-05], [16, 3.8558

# # # # # # # # Usuario: ['Luis Leiva'] Mensaje: servicio de envio [[4, 0.96574], [23, 0.014488988], [17, 0.008702961], [0, 0.004555763], [20, 0.0013680395], [7, 0.0006282937], [11, 0.00059280277], [10, 0.0005754182], [12, 0.0005716475], [22, 0.00052177516], [15, 0.00043608213], [8, 0.00039569684], [21, 0.00027298316], [5, 0.00026877632], [9, 0.00025969063], [13, 0.00018491287], [19, 0.00014453009], [6, 7.889367e-05], [2, 6.8955094e-05], [1, 6.318214e-05], [16, 3.8558577e-05], [3, 1.7211078e-05], [14, 1.3429105e-05], [18, 1.1531787e-05]]
# # # Usuario: ['Luis Leiva'] Mensaje: nacional [[5, 0.98707414], [9, 0.0065842634], [23, 0.0022928326], [21, 0.0012786051], [3, 0.0004643099], [12, 0.00029412855], [7, 0.00028721974], [1, 0.00026441307], [8, 0.00022148716], [16, 0.0002085445], [14, 0.00019965721], [17, 0.00017458743], [10, 0.00017130225], [15, 0.000102100654], [19, 7.815841e-05], [11, 6.435503e-05], [22, 6.094011e-05], [13, 5.5093038e-05], [2, 5.426115e-05], [6, 1.8341374e-05], [4, 1.